# Imports

In [ ]:
!pip install transformers
!pip install datasets


import json
from transformers import AutoTokenizer
from datasets import load_dataset
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
from transformers import Trainer, TrainingArguments
import math
from transformers import AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling



     |████████████████████████████████| 2.0MB 8.3MB/s 
     |████████████████████████████████| 870kB 55.3MB/s 
     |████████████████████████████████| 3.2MB 56.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=b246921c8c4a180a3b0b92b5e645520a2c8b4859cdabbdb1373312e8df60277d
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 194kB 7.5MB/s 
     |████████████████████████████████| 245kB 14.3MB/s 
     |████████████████████████████████| 112kB 15.2MB/s 


# Read in COCO Data

In [ ]:
data_path = './drive/MyDrive/w266_final_project/' 


with open(data_path+'captions_train2017.json') as f:
    train = json.load(f)

with open(data_path+'captions_val2017.json') as f:
    val = json.load(f)

train_corpus = ''
for i in range(len(train['annotations'])):
    train_corpus+=train['annotations'][i]['caption']
    if train_corpus[-1]=='.':
        train_corpus+=' '
    elif train_corpus[-2:]=='. ':
        pass
    else:
        train_corpus+='. '
print(train_corpus[:110])

val_corpus = ''
for i in range(len(val['annotations'])):
    val_corpus+=val['annotations'][i]['caption']
    if val_corpus[-1]=='.':
        val_corpus+=' '
    elif val_corpus[-2:]=='. ':
        pass
    else:
        val_corpus+='. '
print(val_corpus[:110])

A bicycle replica with a clock as the front wheel. A room with blue walls and a white sink and door. A car tha
A black Honda motorcycle parked in front of a garage. A Honda motorcycle parked in a grass driveway. An office


In [ ]:
# clean data and save text files
train_sentences = train_corpus.split(". ")
train_sentences_clean = [i+".\n" for i in train_sentences if len(i.split()) >= 2]
# train_sentences_clean = train_sentences_clean[:30000]

COCO_train_file = open(data_path+"COCO_train.txt","w")
COCO_train_file.writelines(train_sentences_clean)
COCO_train_file.close()

val_sentences = val_corpus.split(". ")
val_sentences_clean = [i+".\n" for i in val_sentences if len(i.split()) >= 2]
# val_sentences_clean = val_sentences_clean[:30000]

COCO_val_file = open(data_path+"COCO_val.txt","w")
COCO_val_file.writelines(val_sentences_clean)
COCO_val_file.close()


In [ ]:
datasets = load_dataset("text", data_files={"train": data_path+"COCO_train.txt", "validation": data_path+"COCO_train.txt"})

Using custom data configuration default-222135261ad2c05e


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-222135261ad2c05e/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


In [ ]:
datasets["train"][10]

{'text': 'An airplane that is, either, landing or just taking off.'}

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,text
0,a snowboarder jumps into the air on his snowboard on a mountain.
1,a giraffe eating out of a tree near bushes.
2,Two people skiing down a small hill in front of a building.
3,A large black looking classic car parked on the grass.
4,Several people gather on a cloudy day to fly kites.
5,A few sheep wander around and drink water.
6,Two men unloading a white refrigerator together outside.
7,The two larger cows are watching over the smaller cows outside.
8,A group of children riding on top of snow covered ground.
9,A kite surfer moving with a strong wind.


# Masked Language Modeling

In [ ]:
model_checkpoint = "bert-base-uncased"

In [ ]:
# Tokenize Data

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,1.451000,1.374656,337.944800,191.614000
2,1.374200,1.306489,337.858700,191.663000
3,1.335300,1.272246,338.024800,191.569000


TrainOutput(global_step=24285, training_loss=1.436194199516323, metrics={'train_runtime': 5119.6896, 'train_samples_per_second': 4.743, 'total_flos': 1.633904263754496e+16, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 356857, 'init_mem_gpu_alloc_delta': 439194112, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 903607, 'train_mem_gpu_alloc_delta': 1346428928, 'train_mem_cpu_peaked_delta': 97358604, 'train_mem_gpu_peaked_delta': 1397728256})

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 3.55


In [ ]:
#save model to use as starting point for subtask A classification

trainer.save_model(data_path+"bert_base_coco")